# Import

In [11]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
pd.options.display.max_columns=999

# Read In Data

In [12]:
# root
folder = './2019/04/14/'

## Aircraft
Has the information about each tracked aircraft. Key metrics can be used:

- **Id**: Aircraft ID
- **AircraftModelId**: Model ID to **join `model` data**
- **Blocked**: Whether the aircraft is blocked by the FAA

In [13]:
aircraft = pd.read_csv(folder + "aircraft_20190414030057.csv")
aircraft = aircraft[["Id", "AircraftModelId", "Blocked"]]
aircraft.head()

,Id,AircraftModelId,Blocked
0,4,121.0,True
1,5,63.0,False
2,9,49.0,False
3,10,46.0,False
4,11,121.0,True


## Model
Has the model information for each aircraft. Key metrics can be used:

(`Crew` is not used here since it has value as large as 48,200, which is not a reasonable number for crew members of a flight)

- **Id**: Can be used to join on **AircraftModelId** in `aircraft` table
- **NormalCruiseSpeed**: Normal cruising speed for the aircraft model
- **NormalRange**: Normal range for the aircraft model
- **NormalPassengers**: Normal passengers for the aircraft model

In [15]:
model = pd.read_csv(folder + "model_20190414030057.csv")
model = model[["Id", "NormalCruiseSpeed", "NormalRange", "NormalPassengers"]]
model.head()

,Id,NormalCruiseSpeed,NormalRange,NormalPassengers
0,121,488.0,5934.0,19.0
1,63,459.0,3340.0,4.0
2,49,470.0,4920.0,15.0
3,46,459.0,3549.0,16.0
4,48,470.0,5882.0,15.0


## Ownership
Has the information about the ownership for each tracked aircraft. Key metrics can be used:

- **AircraftId**: Can be used to join on **Id** in `Aircraft` table
- **CompanyId**: Can be used to join on **Id** in `Company` table
- **OwnershipPercentage**: Ownership percentage held by the company

In [16]:
ownership = pd.read_csv(folder + "ownership_20190414030057.csv")
ownership = ownership[["AircraftId", "CompanyId", "OwnershipPercentage"]]
ownership.head()

,AircraftId,CompanyId,OwnershipPercentage
0,4,1,100.000
1,5,1,9.375
2,5,49488,31.250
3,9,100,12.000
4,9,113,37.000


## Company

Has the information about tracked companies. Key metrics can be used:
- **Id**: Can be used to join on **CompanyId** in `Ownership` table
- **Symbol**: Ticker if the company is public traded, otherwise is empty
- **Industry or Sector**: Indicator of the company's industry or sector
- **Latitude & Longitude**: Geo-coordinate
- **Figi**: For mapping purpose

In [17]:
company = pd.read_csv(folder + "company_20190414030057.csv")
company = company[["Id", "Exchange", "Symbol", "Industry", "Sector", "Latitude", "Longitude", "Figi"]]
company.head()

,Id,Exchange,Symbol,Industry,Sector,Latitude,Longitude,Figi
0,1,NAS,AMZN,Internet Retail,Retail Trade,47.622194,-122.336553,BBG000BVQ4Z3
1,2,NAS,HOLX,Medical Specialties,Health Technology,42.324951,-71.583244,BBG000C3P591
2,3,NYS,UHS,Hospital/Nursing Management,Health Services,40.078556,-75.370639,BBG000CB9T62
3,4,NYS,UTX,Industrial Conglomerates,Producer Manufacturing,41.708447,-72.803224,BBG000BW8TX8
4,5,NYS,VAR,Medical Specialties,Health Technology,37.417032,-122.142878,BBG000BWC1M6


## Relationship
Has the information about the relationship between two companies. Key metrics can be used:

- **CompanyId**: Can be used to join on **Id** in `Company` table
- **RelatedCompanyId**: Can be used to join on **Id** in `Company` table
- **Type**: Type of relationship
- **StartDate**: Beginning of the date range when the relationship was active
- **EndDate**: End of the date range when the relationship was active

In [19]:
relationship = pd.read_csv(folder + "relationships_20190414030057.csv")
relationship = relationship[["CompanyId", "RelatedCompanyId", "Type", "StartDate", "EndDate"]]
relationship.head()

,CompanyId,RelatedCompanyId,Type,StartDate,EndDate
0,1,2080,Partner - In-licensing,2/9/16 12:00:00 AM,2/14/19 6:00:00 AM
1,1,40,Partner - Research Collaboration,2/7/18 12:00:00 AM,2/14/19 6:00:00 AM
2,1,100,Partner - Research Collaboration,1/23/17 12:00:00 AM,NaN
3,1,935,Partner - Research Collaboration,7/27/16 12:00:00 AM,2/14/19 6:00:00 AM
4,1,3721,Partner - In-licensing,2/9/16 12:00:00 AM,2/14/19 6:00:00 AM


## Airport
Has the information about the airports where aircrafts take of or land. Key metrics can be used:

- **Icao**: The ICAO of the airport
- **Classification**: filter out “closed” and use numerical value to show the data instead of categorical
- **Latitude & Longitude**: Can be used to locate the airport and companies nearby
- **Country**: Can be used to indicate overseas or not

In [20]:
airport = pd.read_csv(folder + "airport_20190414030057.csv")
airport = airport[["Icao", "Classification", "Latitude", "Longitude", "Country"]]
airport.head()

,Icao,Classification,Latitude,Longitude,Country
0,00A,heliport,40.070801,-74.933601,US
1,00AA,small_airport,38.704022,-101.473911,US
2,00AK,small_airport,59.949200,-151.695999,US
3,00AL,small_airport,34.864799,-86.770302,US
4,00AR,closed,35.608700,-91.254898,US


## Flight
Has the information about flight hisotry on tracked aircrafts. Key metrics can be used:

- **AircraftId**: Can be used to join on **Id** in `Aircraft` table
- **DepartureTime & ArrivalTime**: Departure and arrival time
- **DepartureIcao & ArrivalIcao**: Can be used to join on **ICao** in `airport` table
- **StayDurationSeconds**: Length of the stay expressed in seconds (Crucial metrics for staying time. Based on the report from JetTrack, a short duration time (one or two hour) indicates a group was dropped off and picked up later. Therefore, the length of the duration could indicate more information than just a time period)

In [21]:
flight = pd.read_csv(folder + "flight_20190414030057.csv", parse_dates=["DepartureTime", "ArrivalTime"])
flight = flight[["AircraftId", "DepartureTime", "DepartureIcao", "ArrivalTime", "ArrivalIcao", "StayDurationSeconds"]].sort_values("DepartureTime").reset_index(drop=True)
flight.head()

,AircraftId,DepartureTime,DepartureIcao,ArrivalTime,ArrivalIcao,StayDurationSeconds
0,679,2000-01-01,KGTF,2015-12-16 23:27:00,KELD,-185765112.0
1,871,2000-01-01,KFTY,2007-03-27 21:09:00,NaN,-7201740.0
2,456,2000-01-01,ZBAA,2011-08-26 09:58:00,PANC,-12779520.0
3,353,2000-01-01,KCSG,2009-04-23 16:35:00,KSAV,-72586155.0
4,537,2000-01-01,KCAK,2013-06-20 21:48:00,KRDG,-52532820.0


## Transactions
Has the information about M&A transactions in different status/stages. Key metrics can be used:
- **Id**: Can be used in filtering
- **CompanyId1**: Can be used to join on **Id** in `Company` table
- **CompanyId2**: Can be used to join on **Id** in `Company` table
- **Status**: Include null, Rumor, Pending, Cancelled, and Complete
- **RumorDate**: The date when the transaction is in Rumor stage
- **CancelDate**: The date when the transaction is cancelled
- **ClosingDate**: The date when the transaction is closed
- **AnnouncementDate**: The date when the transaction is in Announcement stage
- **TargetedClosingDate**: The date when the transaction is expected to be closed

In [10]:
transaction = pd.read_csv(folder + "transactions_20190414030057.csv", parse_dates=["RumorDate", "CancelDate", "ClosingDate", "AnnouncementDate", "TargetedClosingDate"])

# Data Transformation

## Features Part I

In [11]:
# 1. Aircraft: Change blocked values to 0 and 1
aircraft.rename(columns={"Id": "AircraftId"}, inplace=True)
aircraft["Blocked"] = aircraft.apply(lambda x: 1 if x["Blocked"] == True else (0 if x["Blocked"] == False else np.nan), axis=1)

In [12]:
# 2. Merge Aircraft with Model -> air_mod
model.rename(columns={"Id": "AircraftModelId"}, inplace=True)
air_mod = aircraft.merge(model, how="left", on=["AircraftModelId"])

In [13]:
# 3. Merge air_mod with Ownership -> air_mod_own
air_mod_own = air_mod.merge(ownership, how="left", on=["AircraftId"])

In [14]:
# 4. Clean the companies
# - Rename Id to CompanyId for future join
company.rename(columns={"Id": "CompanyId"}, inplace=True)
# - Drop companies without Latitude nor Longitude
company = company[~((company["Latitude"].isnull()) | (company["Longitude"].isnull()))]
# - Drop companies without Industry nor Sector
company = company[~((company["Sector"].isnull()) | (company["Industry"].isnull()))]
# - Drop companies without Symbol since it's impossible to map
company = company[~company["Symbol"].isnull()]
# - Create Ticker = Exchange : Symbol to make each Ticker is unique
company["Ticker"] = company["Exchange"] + ":" + company["Symbol"]
# - Reorder the dataframe
company = company[["CompanyId", "Ticker", "Sector", "Industry", "Latitude", "Longitude", "Figi"]]

In [15]:
# 5. Merge air_mod_own with company -> air_mod_own_comp
air_mod_own_comp = air_mod_own.merge(company, how="left", on="CompanyId")

In [16]:
# 6. Map company to Relationship
comp_rel = relationship.merge(company, how="left", on="CompanyId").merge(company, how="left", left_on="RelatedCompanyId", right_on="CompanyId", suffixes=["_Source", "_Target"])

In [17]:
# 7. Map Airport to Flight -> airport_flight
airport_flight = flight.merge(airport, how="left", left_on="DepartureIcao", right_on="Icao").merge(airport, how="left", left_on="ArrivalIcao", right_on="Icao", suffixes=["_Departure", "_Arrival"])
del airport_flight["DepartureIcao"]
del airport_flight["ArrivalIcao"]
airport_flight["SameCountry"] = airport_flight.apply(lambda x: 1 if x["Country_Departure"] == x["Country_Arrival"] else 0, axis=1)
del airport_flight["Country_Departure"]
del airport_flight["Country_Arrival"]

# Remove negative StayDurationSeconds
airport_flight = airport_flight[airport_flight["StayDurationSeconds"] > 0].reset_index(drop=True)

## Targets Part I

In [18]:
# 1. Remove TargetedClosingDate since it's not meaningful in terms of defining Status
# 2. Remove CancelDate since there are no values
updated_transaction = transaction.drop(["TargetedClosingDate", "CancelDate"], axis=1)

# 3. Remove all records where Status == Cancelled since there are no CancelDate for all Cancelled records
updated_transaction = updated_transaction[updated_transaction["Status"] != "Cancelled"]

# 4. Remove all records where Status == null
updated_transaction = updated_transaction[~updated_transaction["Status"].isnull()]

# 5. Keep the records where Status == Rumor.
"""
- Reason: Might be some insights even thought that is only a rumor
- Notes: 
    - Rumor has the same `RumorDate` and `AnnoucementDate`
    - Didn't observe the `Rumor Cancelled` status, so will assume `Rumor` will last forever
- Remove `Id = 6316`
"""
updated_transaction = updated_transaction[updated_transaction["Id"] != 6316]

# 7. Check if there are duplicated records of the combination of (CompanyId1, CompanyId2)
len(updated_transaction.groupby(["CompanyId1", "CompanyId2"]).size().reset_index().rename(columns={0: "count"}).query("count != 1")) == 0

# 8. There are many transactions where company_1 and company_2 are not in the company table. Filter them out
updated_transaction = updated_transaction[(updated_transaction["CompanyId1"].isin(company["CompanyId"].unique()) & updated_transaction["CompanyId2"].isin(company["CompanyId"].unique()))]

updated_transaction = updated_transaction.reset_index(drop=True)

## Features Part II

In [19]:
# 8. Merge exp_airport_flight with air_mod_own_comp for departure companies (and keep above companies only)
airport_flight_dep_comp = airport_flight.merge(air_mod_own_comp, how="left", on="AircraftId")
cols = ["CompanyId", "Ticker", "Figi", "Sector", "Industry", "DepartureTime", "Classification_Departure"] + \
       [c for c in airport_flight_dep_comp.columns if "Normal" in c] + ["OwnershipPercentage", "Blocked"] + \
       ["ArrivalTime", "Latitude_Arrival", "Longitude_Arrival", "Classification_Arrival"] + \
       ["SameCountry", "StayDurationSeconds"]
exp_airport_flight_dep_comp = airport_flight_dep_comp[cols]

# Further cleaning
exp_airport_flight_dep_comp = exp_airport_flight_dep_comp[~exp_airport_flight_dep_comp["Ticker"].isnull()]
exp_airport_flight_dep_comp = exp_airport_flight_dep_comp[~((exp_airport_flight_dep_comp["Latitude_Arrival"].isnull()) | (exp_airport_flight_dep_comp["Longitude_Arrival"].isnull()))]

# Narrow flight records down to companies in the transaction file
transaction_comp = list(set(updated_transaction["CompanyId1"]) | set(updated_transaction["CompanyId2"]))
narrowed_flights = exp_airport_flight_dep_comp[exp_airport_flight_dep_comp["CompanyId"].isin(transaction_comp)]

# The earliest date in the transaction file is 2009-08-31. Keep records after 2009
narrowed_flights = narrowed_flights.query("ArrivalTime > '2009-01-01'")

# Remove outliers
narrowed_flights = narrowed_flights.query("DepartureTime < ArrivalTime")
narrowed_flights = narrowed_flights.sort_values("ArrivalTime").reset_index(drop=True).reset_index().rename(columns={"index": "flight_id"})

narrowed_flights["TransactionId"] = 0

In [20]:
RADIUS = 100
def distance(lon_a, lat_a, lon_c, lat_c, radius):
    """
    Calculate whether a company is whithin the range of an airport
    for a given radius (in miles)
    """
    lon_a, lat_a, lon_c, lat_c = map(np.radians, [lon_a, lat_a, lon_c, lat_c])

    # haversine formula 
    dlon = lon_c - lon_a 
    dlat = lat_c - lat_a 
    a = np.sin(dlat/2.0)**2 + np.cos(lat_a) * np.cos(lat_c) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 3959 # Radius of earth in miles
    d = c * r
    
    if d <= radius: 
        return True
    else:
        return False

In [21]:
%%time
"""
Based on the transaction records to find qualified flight records
"""
for i in range(len(updated_transaction)):
    comp_id_1 = updated_transaction.iloc[i]["CompanyId1"]
    comp_id_2 = updated_transaction.iloc[i]["CompanyId2"]
    tran_id = updated_transaction.iloc[i]["Id"]
    coords_1 = list(company.query("CompanyId == @comp_id_1")[["Longitude", "Latitude"]].values[0])
    coords_2 = list(company.query("CompanyId == @comp_id_2")[["Longitude", "Latitude"]].values[0])
    df = narrowed_flights.query("CompanyId == @comp_id_1")
    if len(df) != 0:
        for j in df.flight_id.unique():
            coords_arrival = list(df.loc[j][["Longitude_Arrival", "Latitude_Arrival"]].values)
            if distance(coords_arrival[0], coords_arrival[1], coords_2[0], coords_2[1], RADIUS):
                narrowed_flights.loc[j, "TransactionId"] = tran_id
    df = narrowed_flights.query("CompanyId == @comp_id_2")
    if len(df) != 0:
        for j in df.flight_id.unique():
            coords_arrival = list(df.loc[j][["Longitude_Arrival", "Latitude_Arrival"]].values)
            if distance(coords_arrival[0], coords_arrival[1], coords_1[0], coords_1[1], RADIUS):
                narrowed_flights.loc[j, "TransactionId"] = tran_id

CPU times: user 12min 15s, sys: 11.1 s, total: 12min 26s
Wall time: 12min 27s


In [22]:
new_flights = narrowed_flights.query("TransactionId != 0")

# 9. Merge new_flights with company for arrival company
final_new_flights = new_flights.merge(updated_transaction[["Id", "CompanyId2"]], how="left", left_on="TransactionId", right_on="Id").merge(company, how="left", left_on="CompanyId2", right_on="CompanyId", suffixes=["_Source", "_Target"])
final_new_flights.drop(["flight_id", "CompanyId_Source", "Latitude_Arrival", "Longitude_Arrival", "TransactionId", 
                        "Id", "CompanyId2", "CompanyId_Target", "Latitude", "Longitude"], axis=1, inplace=True)

In [23]:
# 10. Map `Relationship` to `new_flights` -> `final_features`
cols = ["Ticker_Source", "Ticker_Target", "Type", "StartDate", "EndDate"]
rel = comp_rel[cols]
final_features = final_new_flights.merge(rel, how="left", on=["Ticker_Source", "Ticker_Target"])

### Notes
- The `fianl_features` has 71,254 records 
- Only 6692 of them have relationship
- The rate of that is very small, so make the `final_features = final_new_flights` at the step 9

In [24]:
final_features = final_new_flights.copy()

# Further cleaning
# Remove the records where Ticker_Source == Ticker_Target
final_features = final_features.query("Ticker_Source != Ticker_Target")
# Remove the records where Ticker_Source is null
final_features = final_features[~final_features["Ticker_Source"].isnull()]
# Remove the records where Ticker_Target is null
final_features = final_features[~final_features["Ticker_Target"].isnull()].reset_index(drop=True)

## Targets Part II

In [25]:
# 8. Merge transaction data with company info
tran_comp = updated_transaction.merge(company[["CompanyId", "Ticker"]], how="left", left_on="CompanyId1", right_on="CompanyId").merge(company[["CompanyId", "Ticker"]], how="left", left_on="CompanyId2", right_on="CompanyId", suffixes=["_Source", "_Target"])
tran_comp = tran_comp[["Ticker_Source", "Ticker_Target", "Status", "RumorDate", "AnnouncementDate", "ClosingDate"]]

# 9. Filter out non-public companies
tran_comp = tran_comp[~tran_comp["Ticker_Source"].isnull()]
tran_comp = tran_comp[~tran_comp["Ticker_Target"].isnull()]

# 10. Remove records where Status == "Complete" but ClosingDate is null
tran_comp["Remove"] = tran_comp.apply(lambda x: 1 if (x["Status"] == "Complete" and str(x["ClosingDate"]) == 'NaT') else 0, axis=1)
tran_comp = tran_comp[tran_comp["Remove"] == 0].reset_index(drop=True)
del tran_comp["Remove"]

# 11. Merge feature with targets
final_df = final_features.merge(tran_comp, how="left")

In [26]:
def status_match(x):
    """
    Find the right status based on the arrival time
    """
    if str(x["Status"]) == "nan":
        return "Nothing"
    elif str(x["Status"]) == "Rumor":
        if x["ArrivalTime"] < x["RumorDate"]:
            return "Nothing"
        else:
            return "Rumor"
    elif str(x["Status"]) == "Pending":
        if str(x["RumorDate"]) != 'NaT' and x["ArrivalTime"] >= x["RumorDate"] and x["ArrivalTime"] < x["AnnouncementDate"]:
            return "Rumor"
        elif x["ArrivalTime"] >= x["AnnouncementDate"]:
            return "Pending"
        else:
            return "Nothing"
    elif str(x["Status"]) == "Complete":
        if str(x["RumorDate"]) != 'NaT' and x["ArrivalTime"] >= x["RumorDate"] and x["ArrivalTime"] < x["AnnouncementDate"]:
            return "Rumor"
        elif x["ArrivalTime"] >= x["AnnouncementDate"] and x["ArrivalTime"] < x["ClosingDate"]:
            return "Pending"
        elif x["ArrivalTime"] >= x["ClosingDate"]:
            return "Complete"
        else:
            return "Nothing"

In [27]:
%%time
# 12. define the new status based on the time
final_df["NewStatus"] = final_df.apply(lambda x: status_match(x), axis=1)
final_df.drop(["Status", "RumorDate", "AnnouncementDate", "ClosingDate"], axis=1, inplace=True)
final_df = final_df.sort_values(["ArrivalTime", "Ticker_Source"]).reset_index(drop=True)

CPU times: user 2.65 s, sys: 34.5 ms, total: 2.68 s
Wall time: 2.69 s


## Write to CSV

### Full Write

In [136]:
final_df.to_csv("jettrack_new.csv", index=False)

### 1000 Samples

In [ ]:
res = pd.concat((final_df[final_df["NewStatus"] != "Nothing"], final_df[final_df["NewStatus"] == "Nothing"].sample(1000)))

In [ ]:
res.to_csv("jettrack_sample_1000.csv")

In [30]:
len(final_df.query("ArrivalTime > '2015-01-01'"))

40944

In [37]:
final_df.query("ArrivalTime > '2015-01-01' and ArrivalTime < '2017-01-01'").NewStatus.unique()

array(['Nothing', 'Pending', 'Complete', 'Rumor'], dtype=object)

In [32]:
final_df.query("ArrivalTime > '2015-01-01'").to_csv("jettrack_new_shrinked.csv", index=False)